Import needed libraries

In [2]:
import requests
import pandas as pd
import numpy as np
import csv
import io
import datetime
import pyodbc
import urllib.request
import json
import pyodbc
import sqlalchemy
from pandas.io.json import json_normalize
import time

ETL raw data from government API at https://data.go.th/ and save as a pickle file

In [4]:
years = {
        '2558' : 'lurONURJJ4a2kCYMt41ZTBuqOCUQvMmp', #used today 
        '2559' : 'aQ92erVTgdvIE7g9zivJGTwOllp0exmR', #used yesterday
        '2560' : 'oAA5nVzZQ6iLTc77L9FXaAyfbS3Z4dpJ', # used for 2559
        '2561' : 'EM5bdWNNsdp7MND4URk76KEWS49yGCMB',
        }
  
#  LMnzeNfoOPRk79eKVpX3e1IWoVtJ2Pfl used
#  NveGgxnickjpTodTKtlo9j319fOHt2eC 


project_list = []
contract_list = []
url = ("https://opend.data.go.th/govspending/cgdcontract?api-key=NveGgxnickjpTodTKtlo9j319fOHt2eC&year=2562&"
       + "budget_start=1000000&limit=100&offset=")
for i in range(0,5):
    start = time.time()
    offset = str(100*i)
    print("getting row number " + offset)
    for trial in [1,2,3,4,5]:
        try:
            response = urllib.request.urlopen(url+offset)
            html = response.read()
            break;
        except:
            print("Something is wrong! Retry at row " + offset + " try again! for the " + str(trial) + " times out of 5 times")

    json_obj = json.loads(html)
    df_temp = pd.DataFrame(json_obj['result'])
    df_contract_temp = pd.json_normalize(data=json_obj['result'], record_path='contract', meta = ['project_id'])
    project_list.append(df_temp)
    contract_list.append(df_contract_temp)
    end = time.time()
    print(str(i) + ' trial takes ' + str(end - start)+ ' seconds')

getting row number 0
0 trial takes 7.318524122238159 seconds
getting row number 100
1 trial takes 4.599499940872192 seconds
getting row number 200
2 trial takes 6.652778387069702 seconds
getting row number 300
3 trial takes 6.790854454040527 seconds
getting row number 400
4 trial takes 5.103748321533203 seconds


In [6]:
df_project = pd.concat(project_list)
df_contract = pd.concat(contract_list)

In [38]:
df_project.project_id.nunique()

82860

In [39]:
print(df_project.shape)
print(df_contract.shape)

(82860, 18)
(90005, 8)


In [40]:
df_project.head()

,project_id,project_name,project_type_name,dept_name,dept_sub_name,purchase_method_group_name,announce_date,project_money,price_build,sum_price_agree,budget_year,transaction_date,province,district,subdistrict,project_status,geom,contract
0,60085110634,เช่าเครื่องและอุปกรณ์เพื่อให้บริการโทรคมนาคม เ...,เช่า,บริษัท ทีโอที จำกัด (มหาชน),ส่วนพัสดุ หน่วยธุรกิจสื่อสารไร้สาย,พิเศษ,-,"97,572,658,000","97,572,658,000","97,572,658,000",2562,5 ธ.ค. 61,กรุงเทพมหานคร,หลักสี่,ทุ่งสองห้อง,ระหว่างดำเนินการ,,"[{'winner_tin': '0105548115901', 'winner': 'บร..."
1,61097014409,ประกวดราคาจ้างก่อสร้างโรงงานผลิตยารังสิต ระยะท...,จ้างก่อสร้าง,องค์การเภสัชกรรม,องค์การเภสัชกรรม(กองจัดซื้อทั่วไป),ประกวดราคาอิเล็กทรอนิกส์ (e-bidding),12 ต.ค. 61,"5,734,344,000","5,490,773,188","5,264,000,000",2562,24 มิ.ย. 62,กรุงเทพมหานคร,ราชเทวี,ทุ่งพญาไท,ระหว่างดำเนินการ,POINT(100.836778879166 14.0580645003046),"[{'winner_tin': '0107545000217', 'winner': 'บร..."
2,61127463277,ประกวดราคาจ้างก่อสร้างงานจ้างเหมาโครงการก่อสร้...,จ้างก่อสร้าง,กรมทางหลวง,กรมทางหลวง กรุงเทพฯ,ประกวดราคาอิเล็กทรอนิกส์ (e-bidding),17 ม.ค. 62,"4,000,000,000","3,998,992,334","3,994,400,000",2562,27 ส.ค. 62,กรุงเทพมหานคร,ราชเทวี,ทุ่งพญาไท,ระหว่างดำเนินการ,POINT(100.387412309647 13.6200794350437),"[{'winner_tin': '0993000413881', 'winner': 'กิ..."
3,62017178375,ประกวดราคาจ้างก่อสร้างงานจ้างเหมาโครงการก่อสร้...,จ้างก่อสร้าง,กรมทางหลวง,กรมทางหลวง กรุงเทพฯ,ประกวดราคาอิเล็กทรอนิกส์ (e-bidding),17 ม.ค. 62,"4,000,000,000","3,998,882,232","3,991,986,000",2562,27 ส.ค. 62,กรุงเทพมหานคร,ราชเทวี,ทุ่งพญาไท,ระหว่างดำเนินการ,POINT(100.365927815437 13.5983352725177),"[{'winner_tin': '0105522017472', 'winner': 'บร..."
4,62037453909,ซื้อโครงการ CAPD และ EPO จำนวน 2 รายการ โดยวิธ...,ซื้อ,โรงพยาบาลราชวิถี,โรงพยาบาลราชวิถี,เฉพาะเจาะจง,-,"3,434,780,026","3,434,780,026","3,434,780,026",2562,1 เม.ย. 62,กรุงเทพมหานคร,ราชเทวี,ทุ่งพญาไท,ระหว่างดำเนินการ,,"[{'winner_tin': '0994000165315', 'winner': 'อง..."


In [41]:
df_project.to_pickle('Raw_project.pkl')
df_contract.to_pickle('Raw_contract.pkl')

In [361]:

project_type_name_th = df.project_type_name.unique()
project_type_name_th = ['จ้างทำของ/จ้างเหมาบริการ', 'จ้างก่อสร้าง', 'เช่า', 'ซื้อ','จ้างที่ปรึกษา']
project_type_name_eng = ['Contracting services / Contracting services', 'Hire for construction',
                         'hire','buy','Hire a consultant']
project_type_name_dict = dict(zip(project_type_name_th,project_type_name_eng))


dept_name_th = ['กองทัพเรือ', 'กองทัพอากาศ', 'การไฟฟ้าส่วนภูมิภาค',
       'การทางพิเศษแห่งประเทศไทย', 'การไฟฟ้าฝ่ายผลิตแห่งประเทศไทย',
       'สำนักงานตำรวจแห่งชาติ', 'กรมทางหลวง',
       'สำนักงานปลัดกระทรวงการคลัง', 'กรมชลประทาน',
       'การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย', 'สถาบันการบินพลเรือน',
       'กรมการขนส่งทางบก', 'กรมท่าอากาศยาน', 'การประปาส่วนภูมิภาค',
       'กรมฝนหลวงและการบินเกษตร', 'สำนักงานหลักประกันสุขภาพแห่งชาติ',
       'สำนักงานปลัดกระทรวงกลาโหม', 'กองบัญชาการกองทัพไทย',
       'สถาบันเทคโนโลยีนิวเคลียร์แห่งชาติ (องค์การมหาชน)',
       'กรมประชาสัมพันธ์', 'กรมบัญชีกลาง']

dept_name_eng = ['Navy', 'Air Force', 'Provincial Electricity Authority',
    'Expressway Authority of Thailand', 'Electricity Generating Authority of Thailand',
    'National Police Agency', 'Department of Highways',
    'Office of the Permanent Secretary for Finance', 'Irrigation Department',
    'Mass Rapid Transit Authority of Thailand', 'Civil Aviation Training Center',
    'Department of Land Transport', 'Department of Airport', 'Provincial Waterworks Authority',
    'Department of Royal Rainmaking and Agricultural Aviation', 'National Health Security Office',
    'Office of the Permanent Secretary for Defense', 'Thai Armed Forces Headquarters',
    'National Institute of Nuclear Technology (Public organization) ',
    'Public Relations Department', 'Comptroller Generals Department']




In [345]:
df_project_temp = df_project.replace(project_type_name_th,['Contracting services / Contracting services', 'Hire for construction',
                         'hire','buy','Hire a consultant'])

In [12]:
df_project_temp = pd.read_pickle('2558_project.pkl')